#### Tool Binding with LLMs 
We will bind the tool for giving more functionality to the LLM. Tool (Tool Node) can be anything like vectorDB. SQLDB or Tavily.

{Start} -> [Chatbot] -> [ToolNode] -> {END}

In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages    # Reducer to append the previous communication back to State.
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=openai_api_key,
    temperature=0.2,
    streaming=True,
)

In [ ]:
from langchain_tavily import TavilySearch

tool=TavilySearch(max_results=2)
# tool.invoke("What is langgraph")

{'query': 'What is langgraph',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
   'title': 'LangGraph Tutorial: What Is LangGraph and How to Use It?',
   'content': 'LangGraph is a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner. By managing the flow of data and the sequence of operations, LangGraph allows developers to focus on the high-level logic of their applications rather than the intricacies of agent coordination. Whether you need a chatbot that can handle various types of user requests or a multi-agent system that performs complex tasks, LangGraph provides the tools to build exactly what you need. LangGraph significantly simplifies the development of complex LLM applications by providing a structured framework for managing state and coordinating agent interaction

In [5]:
## Custom function
def multiply(a:int,b:int)->int:
    """Multiply a and b

    Args:
        a (int): first int
        b (int): second int

    Returns:
        int: output int
    """
    return a*b

In [7]:
tools=[tool,multiply]

In [11]:
# Bind LLM with the tools created
llm_with_tool=llm.bind_tools(tools)
llm_with_tool

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002B7A05D8B30>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002B7A0AB5670>, root_client=<openai.OpenAI object at 0x000002B79F16D070>, root_async_client=<openai.AsyncOpenAI object at 0x000002B7A065F320>, model_name='gpt-4o-mini', temperature=0.2, model_kwargs={}, openai_api_key=SecretStr('**********'), streaming=True), kwargs={'tools': [{'type': 'function', 'function': {'name': 'tavily_search', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. It not only retrieves URLs and snippets, but offers advanced search depths, domain management, time range filters, and image search, this tool delivers real-time, accurate, and citation-backed results.Input should be a search query.', 'parameters': {'properties': {'query': {'descri

In [12]:
## Node definition
def tool_calling_llm(state:State)->State:
    return {"messages":[llm_with_tool.invoke(state["messages"])]}

## Grpah
builder=StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

## Add Edges
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
    {"tools": "tools", END: END},
)
builder.add_edge("tools",END)

## compile the graph
graph=builder.compile()

In [13]:
from IPython.display import Image,display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(e) 

Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


In [19]:
response = graph.invoke({'messages': "What is 5 * 5" })

In [20]:
response['messages'][-1].content

'25'

In [21]:
for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

What is 5 * 5
================================== Ai Message ==================================
Tool Calls:
  multiply (call_qtYjVPYRtzjThxvQT7g03cXq)
 Call ID: call_qtYjVPYRtzjThxvQT7g03cXq
  Args:
    a: 5
    b: 5
================================= Tool Message =================================
Name: multiply

25


### ReAct Agent Architecture

In [22]:
## Node definition
def tool_calling_llm(state:State)->State:
    return {"messages":[llm_with_tool.invoke(state["messages"])]}

## Grpah
reAct_builder=StateGraph(State)
reAct_builder.add_node("tool_calling_llm",tool_calling_llm)
reAct_builder.add_node("tools",ToolNode(tools))

## Add Edges
reAct_builder.add_edge(START, "tool_calling_llm")
reAct_builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
    {"tools": "tools", END: END},
)
reAct_builder.add_edge("tools","tool_calling_llm")

## compile the graph
graph_reAct=reAct_builder.compile()

In [23]:
from IPython.display import Image,display

try:
    display(Image(graph_reAct.get_graph().draw_mermaid_png()))
except Exception as e:
    print(e) 

Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


In [24]:
response = graph_reAct.invoke({'messages': "Give me 5 point news on AI and what is 5*321"})

In [25]:
for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

Give me 5 point news on AI and what is 5*321
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_GPpe1rEv10GbPqWlhgqHZ2UO)
 Call ID: call_GPpe1rEv10GbPqWlhgqHZ2UO
  Args:
    query: latest news on AI
    search_depth: advanced
    topic: news
  multiply (call_Vqjw4MBcZi55DHOKthYiKu4H)
 Call ID: call_Vqjw4MBcZi55DHOKthYiKu4H
  Args:
    a: 5
    b: 321
================================= Tool Message =================================
Name: tavily_search

{"query": "latest news on AI", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://punchng.com/leveraging-ai-to-solve-everyday-challenges/?utm_source=auto-read-also&utm_medium=web", "title": "Leveraging AI to solve everyday challenges - Punch Newspapers", "score": 0.7986339330673218, "published_date": "Tue, 16 Sep 2025 01:15:29 GMT", "content": "* Tuesday, Sept

### Agent with in Memory
This will give the context to the LLM with the help of previous checkpoints. In Memeory is defaultdict which store in form of a thread. Each thread will have stored memory.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver # this will store the memory in threads

# It will store the checkpoint storage.
memory = MemorySaver()

## Node definition
def tool_calling_llm(state:State)->State:
    return {"messages":[llm_with_tool.invoke(state["messages"])]}

## Grpah
reAct_builder_memory=StateGraph(State)
reAct_builder_memory.add_node("tool_calling_llm",tool_calling_llm)
reAct_builder_memory.add_node("tools",ToolNode(tools))

## Add Edges
reAct_builder_memory.add_edge(START, "tool_calling_llm")
reAct_builder_memory.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
    {"tools": "tools", END: END},
)
reAct_builder_memory.add_edge("tools","tool_calling_llm")

## compile the graph
graph_reAct_memory=reAct_builder_memory.compile(checkpointer=memory)

In [ ]:
# Now we need to create a thread Id, to store the intermeidate data/ communication.
config = {"configurable": {"thread_id": "1"}}
# In actual env we will seperately save the configuration (thread id) in storage or distributed system.
response = graph_reAct_memory.invoke({"messages": "My Name is Parag Shah"}, config=config)
response


{'messages': [HumanMessage(content='My Name is Parag Shah', additional_kwargs={}, response_metadata={}, id='fe40c32c-d0c7-4924-8ef3-d8f40f14f819'),
  AIMessage(content='Hello, Parag Shah! How can I assist you today?', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'service_tier': 'default'}, id='run--4fdc6bd3-13c7-4916-a7df-5f2fded9a29d-0')]}

In [30]:
response = graph_reAct_memory.invoke({"messages": "What is my name?"}, config=config)
response

{'messages': [HumanMessage(content='My Name is Parag Shah', additional_kwargs={}, response_metadata={}, id='fe40c32c-d0c7-4924-8ef3-d8f40f14f819'),
  AIMessage(content='Hello, Parag Shah! How can I assist you today?', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'service_tier': 'default'}, id='run--4fdc6bd3-13c7-4916-a7df-5f2fded9a29d-0'),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='0996fc24-6f5a-461c-837f-8c0aeda6e658'),
  AIMessage(content='Your name is Parag Shah.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'service_tier': 'default'}, id='run--d5542e62-e72e-4dba-a686-04196b64abdb-0')]}